# Atmospheric reaction model used by Sandu (2001)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from nodepy import rk
import cvxpy as cp

import plot_fkt
plot_fkt.setup_plt()

import numpy.linalg as linalg
from numba import jit, float64


rk4 = rk.loadRKM('RK44').__num__()
rk4x2 = rk4*rk4
ssp2 = rk.loadRKM('SSP22').__num__()
ssp3 = rk.loadRKM('SSP33').__num__()
ssp104 = rk.loadRKM('SSP104').__num__()
merson4 = rk.loadRKM('Merson43').__num__()
bs5 = rk.loadRKM('BS5').__num__()

trbdf = rk.loadRKM('TR-BDF2').__num__()
be = rk.loadRKM('BE').__num__()
irk2 = rk.loadRKM('LobattoIIIA2').__num__()

ck5 = rk.loadRKM('CK5').__num__()
dp5 = rk.loadRKM('DP5').__num__()
pd8 = rk.loadRKM('PD8').__num__()

ex2 = rk.extrap(2,'implicit euler').__num__()
ex3 = rk.extrap(3,'implicit euler').__num__()

from OrderCondition import *
from RKimple import *
import utils

In [ ]:
ex3 = rk.extrap(3,'implicit euler').__num__()
ex3.bhat=np.array([-1,1,1,0,0,0])

In [ ]:
@jit(float64(float64),nopython=True)
def sigma(t):
    t_start = 12 #To make it start at noon
    T = ((t/3600)+t_start)%24
    Tr = 4.5
    Ts = 19.5

    if Tr <= T <= Ts:
        return 0.5+0.5*np.cos(np.pi*np.abs((2*T-Tr-Ts)/(Ts-Tr))*((2*T-Tr-Ts)/(Ts-Tr)))
    else:
        return 0


name = ['O1D','O','O3','O2','NO','NO2']
name_tex = ['$O^{1D}$','$O$','$O_3$','$O_2$','$NO$','$NO_2$']


@jit(float64[:](float64,float64[:]),nopython=True)
def f_stratospheric_reac(t,u):
    sig = sigma(t)
    
    [O1D,O,O3,O2,NO,NO2] =u
    
    k1 = 2.643e-10*sig**3
    r1 = O2*k1

    k2 = 8.018e-17
    r2 = O*O2*k2

    k3 = 6.120e-4*sig
    r3 = O3*k3

    k4 = 1.567e-15
    r4 = O3*O*k4

    k5 = 1.070e-3*sig**2
    r5 = O3*k5

    M  = 8.120e16
    k6= 7.110e-11
    r6= O1D*M*k6

    k7 = 1.200e-10
    r7 = O1D*O3*k7

    k8 = 6.062e-15
    r8 = O3*NO*k8

    k9 = 1.069e-11
    r9 = NO2*O*k9

    k10=1.289e-2*sig
    r10=NO2*k10

    k11=1.0e-8
    r11=NO*O*k11
    
    dO1D =                      +r5 -r6 -  r7         
    dO   =  +2*r1 -r2 +r3 -  r4     +r6           -r9 +r10 -r11
    dO3  =        +r2 -r3 -  r4 -r5     -  r7 -r8    
    dO2  =  -  r1 -r2 +r3 +2*r4 +r5     +2*r7 +r8 +r9
    dNO  =                                    -r8 +r9 +r10 -r11
    dNO2 =                                    +r8 -r9 -r10 +r11
    
    return np.array([dO1D,dO,dO3,dO2,dNO,dNO2])

In [ ]:
u0 = np.array([9.906e+1,6.624e08,5.326e11,1.697e16,4.000e6,1.093e9])

scale = u0

@jit(float64[:](float64,float64[:]),nopython=True)
def f_stratospheric_reac_scaled(t,u):
    return f_stratospheric_reac(t,u*scale)/scale

# linear invariants
sum_o = np.array([1,1,3,2,1,2])
sum_n = np.array([0,0,0,0,1,1])

In [ ]:
# step size control
beta1 = (0.7/5)
beta2 = (0.3/5)

#compute a total error out of error and change and use this as input for a single PI controller
def dt_logic_PI(stepsize_control,dt_old,dt_adp,error,change,success,tol_met):
    print('error:',error)

    facmax = 1.2
    facmin = 0.1
    fac = 10
    w_change = 1
    #compute total error
    if change == None:
        change = 10  #maybe set her some other value
    error_tot = error + w_change*change
    
    dt_old = dt_logic_PI.dt_old 

    #Control
    Tol = stepsize_control.a_tol
    dt = dt_old *(Tol/error_tot)**beta1*(dt_logic_PI.error_tot_old[-1]/error_tot)**beta2
    
    #Update storage vaiables
    dt_logic_PI.error_tot_old[:-1] = dt_logic_PI.error_tot_old[1:] 
    dt_logic_PI.error_tot_old[-1] = error_tot
    
    
    print('Error/Tol:',error_tot/Tol)
    print('dt_suggest:',dt)
    #Correct for negative values
    if success:
        fac_pos = facmax
    else:
        fac_pos = 0.5
    
    dt = min(facmax*dt_old,fac_pos*dt_old,max(facmin*dt_old,dt))
    dt_logic_PI.dt_old=dt
    return max(dt,stepsize_control.dt_min)


dt_logic_PI.error_tot_old = np.zeros(3)
dt_logic_PI.dt_old=300

In [ ]:
problem = Problem(f=f_stratospheric_reac,
                 u0=u0,
                 minval=0,
                 maxval=np.inf)

problem_scaled = Problem(f=f_stratospheric_reac_scaled,
                 u0=np.ones_like(scale),
                 minval=0,
                 maxval=np.inf)

In [ ]:
dt_logic_PI.error_tot_old = np.zeros(3)
dt_logic_PI.dt_old=1

solver_ref = Solver(rkm = ex3,
               dt = 0.1,
               t_final = 3600*72,
               b_fixed=True,
               tol_neg=1,
               tol_change = np.inf,
               p = [3,2,1],
               theta = [1],
               solver = cp.MOSEK,
               convex=False,
               solver_eqs=solver_nonlinear_arg,
               LP_opts = {'reduce':True},
               fail_on_requect=False)



control_ref = StepsizeControl(dt_min = 0.1,dt_max = np.infty,a_tol = 0.000001,r_tol=1,f = dt_logic_PI
                          ,tol_reqect = 0.0001)


status_ref,t_ref,u_ref,b_ref = RK_integrate(solver=solver_ref,problem=problem_scaled,
                                            stepsize_control=control_ref,verbose=True,dumpK=False)

t_ref = np.array(t_ref)
u_ref = np.array(u_ref).T
utils.show_status(status_ref)

In [ ]:
dt_logic_PI.error_tot_old = np.zeros(3)
dt_logic_PI.dt_old=1

solver = Solver(rkm = ex3,
               dt = 0.1,
               t_final = 3600*72,
               b_fixed=False,
               tol_neg=1,
               tol_change = np.inf,
               p = [3,2,1],
               theta = [1],
               solver = cp.MOSEK,
               convex=False,
               solver_eqs=solver_nonlinear_arg,
               LP_opts = {'reduce':True},
               fail_on_requect=False)



control = StepsizeControl(dt_min = 0.1,dt_max = np.infty,a_tol = 0.015,r_tol=1,f = dt_logic_PI
                          ,tol_reqect = 0.5)


status,t,u,b = RK_integrate(solver=solver,problem=problem_scaled,stepsize_control=control,verbose=True,dumpK=False)

t = np.array(t)
u = np.array(u).T
b = np.array(b).T
utils.show_status(status)

In [ ]:
s = scale.copy() # we have to cahnge the shape
s.shape=[6,1]
u_scaled=u*s
u_scaled_ref=u_ref*s


#unshift t and convert to h
t_h = t/3600+12
t_h_ref = t_ref/3600+12
plt.figure(figsize=[12, 8])
plt.subplot(3,2,1)
i=3
plt.plot(t_h_ref,u_scaled_ref[i,:],color='C1',label='reference')
plt.plot(t_h,u_scaled[i,:],color='C0',label='adapted BE 3 extrap.')
plt.ylabel(name_tex[i])
plt.grid()
plt.gca().tick_params(labelbottom=False)    
plt.legend()

plt.subplot(3,2,3)
i=2
plt.plot(t_h_ref,u_scaled_ref[i,:],color='C1')
plt.plot(t_h,u_scaled[i,:],color='C0')
plt.ylabel(name_tex[i])
plt.grid()
plt.gca().tick_params(labelbottom=False)    


plt.subplot(3,2,5)
i=0
plt.plot(t_h_ref,u_scaled_ref[i,:],color='C1')
plt.plot(t_h,u_scaled[i,:],color='C0')
plt.xlabel('$t$ [h]')
plt.ylabel(name_tex[i])
plt.grid()



plt.subplot(3,2,2)
i=1
plt.plot(t_h_ref,u_scaled_ref[i,:],color='C1')
plt.plot(t_h,u_scaled[i,:],color='C0')
plt.ylabel(name_tex[i])
plt.grid()
plt.gca().tick_params(labelbottom=False)    


plt.subplot(3,2,4)
i=4
plt.plot(t_h_ref,u_scaled_ref[i,:],color='C1')
plt.plot(t_h,u_scaled[i,:],color='C0')
plt.ylabel(name_tex[i])
plt.grid()
plt.gca().tick_params(labelbottom=False)    

plt.subplot(3,2,6)
i=5
plt.plot(t_h_ref,u_scaled_ref[i,:],color='C1')
plt.plot(t_h,u_scaled[i,:],color='C0')
plt.ylabel(name_tex[i])
plt.grid()

plt.tight_layout()
plt.xlabel('$t$ [h]')

plt.savefig('Stratospheric_time.pdf',bbox_inches = "tight")

In [ ]:
rkm = ex3

C0_p="#815900"
C1_p="#306541"

plt.figure(figsize=[6.4*2, 5.1*2])
dt = np.array(status['dt_calc'])
old_min = np.array(status['old_min'])
new_min = np.array(status['new_min'])

change = np.array(status['change'])
error = np.array(status['error'])    
sc= np.array(status['sc'])=='m'


plt.subplot(6,1,1)
plt.plot(t_h[sc],dt[sc],'+C0')
plt.plot(t_h[~sc],dt[~sc],'P',color=C0_p)
plt.ylabel('$\Delta t$')
plt.grid()
plt.xlim([11,85])
plt.gca().tick_params(labelbottom=False)    

plt.subplot(6,1,2)
plt.plot(t_h[sc],old_min[sc],'xC0',label='before adaptation')
plt.plot(t_h[~sc],old_min[~sc],'X',color=C0_p)
plt.plot(t_h[sc],new_min[sc],'+C1',label='after adaptation')
plt.plot(t_h[~sc],new_min[~sc],'P',color=C1_p)
plt.grid()
plt.xlim([11,85])
ax = plt.gca()
plt.legend(loc=5)
plt.ylabel(r'$\min(u_i)$')
plt.gca().tick_params(labelbottom=False)    

plt.subplot(6,1,3)
plt.plot(t_h[sc],error[sc],'xC0',label = '$err_{T}$')
plt.plot(t_h[~sc],error[~sc],'X',color=C0_p)
plt.plot(t_h[sc],change[sc],'+C1',label = r'$\delta$')
plt.plot(t_h[~sc],change[~sc],'P',color=C1_p)
plt.xlim([11,85])
plt.grid()
plt.legend(loc=5)
plt.ylabel('Error')
plt.gca().tick_params(labelbottom=False)    


plt.subplot(6,1,4)
b_orig = rkm.b.copy()
b_orig.shape=(len(b_orig),1)
plt.plot(t_h[1:][sc[1:]],np.linalg.norm((b-b_orig),axis=0,ord=1)[1:][sc[1:]],'xC1')
plt.plot(t_h[1:][~sc[1:]],np.linalg.norm((b-b_orig),axis=0,ord=1)[1:][~sc[1:]],'X',color=C1_p)
plt.grid()
plt.xlim([11,85])
plt.ylabel(r'$\| \tilde{b} - b \|_1 $')
plt.gca().tick_params(labelbottom=False)    



o_ref=sum_o@u_scaled[:,1]
plt.subplot(6,1,5)
plt.plot(t_h[1:][sc[1:]],((sum_o@u_scaled)[1:][sc[1:]]-o_ref)/o_ref,'+C0',label=r'$\sigma_O$')
plt.grid()
plt.xlim([11,85])
ax = plt.gca()
ax.set_yticks((-5e-16,0,5e-16))
ax.set_yticklabels((r'$-5 \times 10^{-16}$', r'$0$', r'$5 \times 10^{-16}$'))
plt.ylabel(r'$\frac{m_O^T u -m_O^T u(0)}{m_O^T u(0)}$')
plt.gca().tick_params(labelbottom=False)    



n_ref=sum_n@u_scaled[:,1]
plt.subplot(6,1,6)
plt.plot(t_h[1:][sc[1:]],((sum_n@u_scaled)[1:][sc[1:]]-n_ref)/n_ref,'+C0',label=r'$\sigma_N$')
plt.grid()
plt.xlim([11,85])
ax = plt.gca()
ax.set_yticks((-5e-16,0,5e-16))
ax.set_yticklabels((r'$-5 \times 10^{-16}$', r'$0$', r'$5 \times 10^{-16}$'))
plt.ylabel(r'$\frac{m_N^T u -m_N^T u(0)}{m_N^T u(0)}$ ')


plt.xlabel('$t$ [h]')

plt.savefig('Stratospheric_stepsize_b.pdf',bbox_inches = "tight")
""" Thick x and + are for values that got rejected for err>tol"""

In [ ]:
sc= np.array(status['sc'])=='m'

old_min = np.array(status['old_min'])
new_min = np.array(status['new_min'])

old_min[old_min == None] = 0
new_min[new_min == None] = 0
#Stats:
bs=np.array(status['b'])
print('Number of steps:',np.sum(bs!=None))
print('Number of adaptions:',np.sum(bs=='c'))

print('Number of rejected steps',np.sum(~sc[1:]))
print('Number of steps where most negative number got worse:',np.sum((old_min > new_min)&sc))
print('Number of steps where most negative number got better:',np.sum((old_min < new_min)&sc))
print('Number of steps with significant change of most negative number:',np.sum((old_min < new_min-1e-20)&sc))

In [ ]:
print('smallest value of u:',np.min(u))
print('smallest value of u_ref:',np.min(u_ref)) #this is done using a far lower tolerance!

In [ ]:
o_ref=sum_o@u_scaled[:,1]
plt.plot((sum_o@u_scaled-o_ref)/o_ref)
n_ref=sum_n@u_scaled[:,1]
plt.plot((sum_n@u_scaled-n_ref)/n_ref)

In [ ]:
print('sum O=',sum_o@u_scaled[:,1])
print('sum N=',sum_n@u_scaled[:,1])

In [ ]:
np.max(t_h)

In [ ]:
np.min(u_ref)

In [ ]:
display(new_min[(old_min < new_min)&sc])
display(old_min[(old_min < new_min)&sc])

In [ ]:
print(utils.get_max_iter_h(status))